In [1]:

import os, sys, subprocess, time, toml
import pandas as pd
from MOASMO_parameters import generate_initial_parameter_sets, surrogate_model_train_and_pareto_points
import run_multiple_paramsets_Derecho

########################################################################################################################
# load configurations

config_file = '/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO/configuration/_level1-0_config_MOASMO.toml'
config = toml.load(config_file)
iter_end = 1

In [2]:

# inputs
file_parameter_list = config['file_calib_param']
path_CTSM_base = config['path_CTSM_case']
path_script_MOASMO = config['path_script_MOASMO']
path_CTSM_source = config['path_CTSM_source']
ref_streamflow = config['file_Qobs']

if 'add_flow_file' in config:
    add_flow_file = config['add_flow_file']
else:
    add_flow_file = 'NA'

script_singlerun = f'{path_script_MOASMO}/run_one_paramset_Derecho.py'
script_clone = f'{path_CTSM_source}/cime/scripts/create_clone'


# outputs
if config['path_calib'] == 'NA':
    path_MOASMOcalib = f'{path_CTSM_base}_MOASMOcalib'
else:
    path_MOASMOcalib = config['path_calib']
    
path_paramset = f'{path_MOASMOcalib}/param_sets'
path_submit = f'{path_MOASMOcalib}/run_model'
path_archive = f'{path_MOASMOcalib}/ctsm_outputs'
os.makedirs(path_MOASMOcalib, exist_ok=True)


# MO-ASMO parameters
sampling_method = config['sampling_method']
num_init = config['num_init'] # initial number of samples
num_per_iter = config['num_per_iter'] # number of selected pareto parameter sets for each iteration
num_iter = config['num_iter'] # including the initial iteration

# evaluation period
RUN_STARTDATE = config['RUN_STARTDATE']
ignore_month = config['ignore_month']
STOP_OPTION = config['STOP_OPTION']
STOP_N = config['STOP_N']

if 'nonstandard_evaluation' in config:
    nonstandard_evaluation = config['nonstandard_evaluation']
else:
    nonstandard_evaluation = 'NA'

# HPC job settings
job_mode = config['job_mode']
job_CTSMiteration = config['job_CTSMiteration']
# job_controlMOASMO = config['job_controlMOASMO'] # not needed here

date_start = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=ignore_month)).strftime('%Y-%m-%d') # ignor the first year when evaluating model
if STOP_OPTION == 'nyears':
    date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(years=STOP_N)).strftime('%Y-%m-%d')
elif STOP_OPTION == 'nmonths':
    date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=STOP_N)).strftime('%Y-%m-%d')
else:
    sys.exit(f'STOP_OPTION must be nyears or nmonths. {STOP_OPTION} is not accepted.')


In [3]:
file_metric_all = []
file_param_all = []

t1 = time.time()
for it in range(0, iter_end):
    print('#'*50)
    print(f'Start iterattion {it}. Total iteration number: {num_iter}')

    iterflag = it

    if it == 0:
        sample_num = num_init
    else:
        sample_num = num_per_iter

    file_metric_iter, file_param_iter = run_multiple_paramsets_Derecho.check_if_all_runs_are_finsihed(path_archive, iterflag, sample_num)
    file_metric_all.append(file_metric_iter)
    file_param_all.append(file_param_iter)

# train a surrogate model and select pareto parameter sets
surrogate_model_train_and_pareto_points(file_parameter_list, file_param_all, file_metric_all, path_paramset, iterflag, num_per_iter, path_CTSM_base)

# generate submission commands (note, this won't submit a real job on Derecho)
run_multiple_paramsets_Derecho.generate_and_submit_multi_CTSM_runs(iter_end, path_submit, path_paramset, path_CTSM_base, 
                                                                   path_archive, script_singlerun, script_clone, 
                                                                   date_start, date_end, ref_streamflow, add_flow_file,
                                                                   job_CTSMiteration, job_mode)


t2 = time.time()
print(f'Iteration {it} is complete. Time cost (s) is {t2 - t1}')

##################################################
Start iterattion 0. Total iteration number: 16
GPR CV KGE Score for metric1/metric2:


,Fold,kge1,kge2,kge_mean
0,1,0.877216,-20.192240,-9.657512
1,2,0.835489,0.708413,0.771951
2,3,0.852215,0.755890,0.804052
3,4,0.832827,0.709957,0.771392
4,5,0.836591,0.633923,0.735257
5,mean,0.846868,-3.476811,-1.314972


RF CV KGE Score for metric1/metric2:


,Fold,kge1,kge2,kge_mean
0,1,0.725574,0.653785,0.689679
1,2,0.680498,0.550630,0.615564
2,3,0.737660,0.718680,0.728170
3,4,0.767889,0.753641,0.760765
4,5,0.686156,0.625665,0.655911
5,mean,0.719555,0.660480,0.690018


Use RF model
Load default parameter values from: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/all_default_parameters.pkl
Generating parameter file: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter1_trial0.pkl
Generating parameter file: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter1_trial1.pkl
Generating parameter file: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter1_trial2.pkl
Generating parameter file: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter1_trial3.pkl
Generating parameter file: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter1_trial4.pkl
Generating parameter file: /g